In [35]:
import pandas_datareader as pdr
from datetime import datetime
import json
from os import listdir
import pandas as pd

In [2]:
start_date = datetime(1970, 1, 1)
end_date = datetime.now().date()
tickers_path = 'data/tickers/sp500.json'
dst_path = 'data/stocks/'

In [47]:
tickers_path = 'data/tickers/test.json'
dst_path = 'data/stocks/test/'

In [50]:
def download_data(tickers_path, dst_path, start_date=datetime(1970, 1, 1), end_date=datetime.now().date(), columns=['Close']):

    assert 'Close' in columns

    with open(tickers_path, 'r') as f:
        tickers = json.load(f)

    success = []
    fail = []

    for ticker in tickers:
        
        ticker = ticker.replace(".", "-")

        try:
            df = pdr.get_data_yahoo(symbols=ticker, start=start_date, end=end_date)

            drop_list = list(set(df.columns) - set(columns))
            df.drop(drop_list,axis=1,inplace=True)


            download_path = dst_path + ticker + '.csv'
            df.to_csv(download_path, index=True)

            success.append(ticker)
        except:
            fail.append(ticker)

    return success, fail

In [64]:
def merge_csv(src_path, dst_path, filename='dataset', how='outer'):
    
    files = [src_path + x for x in listdir(src_path) if 'csv' in x]

    df1 = pd.read_csv(files[0])
    df1 = df1.rename({'Close': files[0].replace(src_path,'').replace('.csv','')}, axis=1)

    for i in range(1,len(files)):
        df2 = pd.read_csv(files[i])
        df2 = df2.rename({'Close': files[i].replace(src_path,'').replace('.csv','')}, axis=1)

        df1 = pd.merge(df1, df2, how=how, on=['Date','Date']) 

    df1['Date'] = pd.to_datetime(df1['Date'])
    df1 = df1.sort_values(by='Date')

    df1 = df1.set_index('Date')
    
    df1.to_csv(dst_path + filename + '.csv')

    return df1